In [ ]:
import os
import io
import base64
import sqlite3
import tempfile
from typing import List, Optional, Dict, Any
from datetime import datetime
import re
import warnings

# Third-party imports
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from IPython.display import Markdown, display, HTML
import PyPDF2 
import docx  
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Suppress warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Initialize Gemini model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.7,
    max_tokens=4000,
)

# Create output schema for topic recommendations
class RecommendedTopic(BaseModel):
    topic: str = Field(description="The name of the recommended topic")
    description: str = Field(description="A brief description of why this topic is relevant")
    resource_url: str = Field(description="A relevant resource URL for this topic")

class TopicRecommendations(BaseModel):
    recommendations: List[RecommendedTopic] = Field(description="List of recommended related topics")

# Create output schema for paper recommendations
class RecommendedPaper(BaseModel):
    title: str = Field(description="The title of the recommended research paper")
    authors: str = Field(description="The authors of the paper")
    year: str = Field(description="Publication year")
    description: str = Field(description="Brief description of relevance to the original paper")
    paper_url: str = Field(description="URL to access this paper", default="")

class PaperRecommendations(BaseModel):
    recommendations: List[RecommendedPaper] = Field(description="List of recommended related papers")

# Create prompt templates
report_prompt = ChatPromptTemplate.from_template(
    """
    You are an AI research assistant. Create a comprehensive, detailed report on the following topic:
    
    Topic: {topic}
    
    Your report should include:
    1. Introduction to the topic
    2. Key concepts and definitions
    3. Historical context and development
    4. Current state and applications
    5. Future directions and potential developments
    6. Conclusion
    
    Format your report with clear markdown headings and subheadings. Use proper markdown formatting for emphasis, lists, and other elements.
    Make sure to provide in-depth analysis.
    """
)

recommendation_prompt = ChatPromptTemplate.from_template(
    """
    Based on the topic: {topic}
    
    Generate 5 relevant related topics that the user might be interested in researching next.
    For each recommendation, provide:
    1. The topic name
    2. A brief 1-2 sentence description of why it's relevant
    3. A relevant resource URL that would contain valuable information about this topic
    
    Your response must be formatted as a valid JSON object that matches this structure:
    {
        "recommendations": [
            {
                "topic": "Topic Name",
                "description": "Brief description of relevance",
                "resource_url": "https://example.com/relevant-page"
            },
            ...
        ]
    }
    
    Use reputable sources for your resource URLs. While you can't verify if the exact URLs exist,
    make them realistic and likely to contain quality information.
    """
)

paper_summary_prompt = ChatPromptTemplate.from_template(
    """
    You are an AI research assistant. Create a concise but comprehensive summary of the following research paper:
    
    Paper content: {paper_content}
    
    Your summary should include:
    1. Main objective of the research
    2. Methodology used
    3. Key findings and results
    4. Main conclusions and implications
    5. Limitations (if mentioned)
    
    Format your summary with clear markdown headings and keep it concise yet informative.
    Focus on the most important aspects of the paper.
    """
)

paper_recommendation_prompt = ChatPromptTemplate.from_template(
    """
    Based on the following research paper:
    
    Paper content: {paper_content}
    
    Generate 5 relevant related research papers that the user might be interested in reading next.
    These should be real papers that likely exist in the academic literature.
    
    For each recommendation, provide:
    1. The paper title (use the actual title of a real paper if you know it)
    2. The authors (use "et al." for multiple authors after the first)
    3. Publication year (estimate if necessary)
    4. A brief description of why it's relevant to the original paper
    5. A URL where the paper might be found - THIS IS CRITICAL. 
    
    For URLs, use specific links from:
    - Google Scholar (https://scholar.google.com/scholar?q=PAPER_TITLE)
    - arXiv (https://arxiv.org/search/?query=PAPER_TITLE)
    - ResearchGate (https://www.researchgate.net/search.Search.html?query=PAPER_TITLE)
    - ACM Digital Library (https://dl.acm.org/action/doSearch?AllField=PAPER_TITLE)
    - IEEE Xplore (https://ieeexplore.ieee.org/search/searchresult.jsp?queryText=PAPER_TITLE)
    
    Replace PAPER_TITLE with URL-encoded paper title in these templates. Make sure EVERY recommendation has a working URL.
    
    Your response must be formatted as a valid JSON object that matches this structure:
    {{
        "recommendations": [
            {{
                "title": "Paper Title",
                "authors": "Author names",
                "year": "Publication year",
                "description": "Brief description of relevance",
                "paper_url": "https://example.com/paper-link"
            }},
            ...
        ]
    }}
    """
)

# Create chains
report_chain = (
    {"topic": RunnablePassthrough()}
    | report_prompt
    | gemini_model
    | StrOutputParser()
)

recommendation_chain = (
    {"topic": RunnablePassthrough()}
    | recommendation_prompt
    | gemini_model
    | JsonOutputParser(pydantic_object=TopicRecommendations)
)

paper_summary_chain = (
    {"paper_content": RunnablePassthrough()}
    | paper_summary_prompt
    | gemini_model
    | StrOutputParser()
)

paper_recommendation_chain = (
    {"paper_content": RunnablePassthrough()}
    | paper_recommendation_prompt
    | gemini_model
    | JsonOutputParser(pydantic_object=PaperRecommendations)
)

# --------------------- Database Functions ---------------------
def initialize_database(db_path: str = "research_papers.db"):
    """Initialize SQLite database for storing papers"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS papers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            content TEXT NOT NULL,
            file_type TEXT NOT NULL,
            upload_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            summary TEXT
        )
    ''')
    conn.commit()
    conn.close()

def save_file_to_database(filename: str, content: str, file_type: str, db_path: str = "research_papers.db"):
    """Save file content to SQLite database"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO papers (filename, content, file_type) VALUES (?, ?, ?)",
        (filename, content, file_type)
    )
    paper_id = cursor.lastrowid
    conn.commit()
    conn.close()
    return paper_id

def save_summary_to_database(paper_id: int, summary: str, db_path: str = "research_papers.db"):
    """Save paper summary to database"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE papers SET summary = ? WHERE id = ?",
        (summary, paper_id)
    )
    conn.commit()
    conn.close()

def get_paper_from_database(paper_id: int, db_path: str = "research_papers.db"):
    """Retrieve paper content from database"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT filename, content, file_type, summary FROM papers WHERE id = ?", (paper_id,))
    result = cursor.fetchone()
    conn.close()
    if result:
        return {
            "filename": result[0],
            "content": result[1],
            "file_type": result[2],
            "summary": result[3]
        }
    else:
        return None

# --------------------- Text Extraction Functions ---------------------
def extract_text_from_pdf(file_path: str) -> str:
    """Extract text content from a PDF file"""
    try:
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        text_content = "\n\n".join([doc.page_content for doc in documents])
        return text_content
    except Exception as e:
        print(f"Error extracting text from PDF: {str(e)}")
        # Fallback method
        text = ""
        with open(file_path, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text

def extract_text_from_docx(file_path: str) -> str:
    """Extract text content from a DOCX file"""
    try:
        loader = Docx2txtLoader(file_path)
        documents = loader.load()
        text_content = "\n\n".join([doc.page_content for doc in documents])
        return text_content
    except Exception as e:
        print(f"Error extracting text from DOCX: {str(e)}")
        # Fallback method
        doc = docx.Document(file_path)
        text = ""
        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text

# --------------------- Report and Recommendation Functions ---------------------
def generate_report(topic: str) -> str:
    """Generate a detailed report on the given topic"""
    return report_chain.invoke(topic)

def generate_recommendations(topic: str) -> str:
    """Generate relevant topic recommendations using Gemini API"""
    try:
        recommendations_data = recommendation_chain.invoke(topic)
        formatted_recommendations = "# Related Topics You May Be Interested In\n\n"
        for i, rec in enumerate(recommendations_data.recommendations, 1):
            formatted_recommendations += f"## {i}. {rec.topic}\n"
            formatted_recommendations += f"{rec.description}\n"
            formatted_recommendations += f"[Learn more]({rec.resource_url})\n\n"
        return formatted_recommendations
    except Exception as e:
        backup_prompt = ChatPromptTemplate.from_template(
            """
            Based on the topic: {topic}
            
            Provide 5 relevant related topics that the user might be interested in researching next.
            For each recommendation, provide:
            1. The topic name
            2. A brief description of why it's relevant
            3. A relevant resource link
            
            Format your response as a markdown list.
            """
        )
        backup_chain = backup_prompt | gemini_model | StrOutputParser()
        return backup_chain.invoke({"topic": topic})

def create_full_report(topic: str, report_content: str, recommendations_content: str) -> str:
    """Create a full markdown report combining the report and recommendations"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    full_report = f"""
# Research Report: {topic}

*Generated on: {timestamp}*

---

{report_content}

---

{recommendations_content}

---

*This report was generated by AI Research Assistant using Gemini API*
"""
    return full_report

def create_full_paper_analysis(filename: str, summary_content: str, recommendations_content: str) -> str:
    """Create a full markdown report for paper analysis"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    full_report = f"""
# Research Paper Analysis: {filename}

*Generated on: {timestamp}*

---

## Paper Summary

{summary_content}

---

{recommendations_content}

---

*This analysis was generated by AI Research Assistant using Gemini API*
"""
    return full_report

# --------------------- File and Display Functions ---------------------
def sanitize_filename(filename: str) -> str:
    """Convert a string to a valid filename"""
    return re.sub(r'[\\/*?:"<>|]', "_", filename)

def save_markdown_file(topic: str, content: str) -> str:
    """Save content to a markdown file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_topic = sanitize_filename(topic)
    filename = f"research_{safe_topic}_{timestamp}.md"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)
    return filename

def display_markdown(content: str, use_markdown_display: bool = True):
    """Display content as rendered markdown if in IPython environment"""
    try:
        if use_markdown_display:
            display(Markdown(content))
        else:
            print(content)
    except:
        print(content)

# --------------------- Paper Processing Functions ---------------------
def process_research_paper(file_path: str, original_filename: Optional[str] = None, db_path: str = "research_papers.db") -> Dict[str, Any]:
    """Process a research paper file (PDF or DOCX)"""
    if not original_filename:
        original_filename = os.path.basename(file_path)
    file_extension = os.path.splitext(original_filename)[1].lower()
    if file_extension == '.pdf':
        text_content = extract_text_from_pdf(file_path)
        file_type = 'pdf'
    elif file_extension in ['.docx', '.doc']:
        text_content = extract_text_from_docx(file_path)
        file_type = 'docx'
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=12000,
        chunk_overlap=2000
    )
    chunks = text_splitter.split_text(text_content)
    processing_text = chunks[0] if len(chunks) > 0 else text_content
    paper_id = save_file_to_database(original_filename, text_content, file_type, db_path=db_path)
    try:
        print("- Generating research paper summary...")
        summary = paper_summary_chain.invoke(processing_text)
        save_summary_to_database(paper_id, summary, db_path=db_path)
        print("- Finding related research papers with access links...")
        try:
            recommendations_data = paper_recommendation_chain.invoke(processing_text)
            recs = recommendations_data.get("recommendations", [])
            formatted_recommendations = "# Related Research Papers You May Be Interested In\n\n"
            for i, rec in enumerate(recs, 1):
                formatted_recommendations += f"## {i}. {rec['title']} ({rec['year']})\n"
                formatted_recommendations += f"**Authors:** {rec['authors']}\n\n"
                formatted_recommendations += f"{rec['description']}\n"
                paper_url = rec.get("paper_url", "").strip()
                if not paper_url:
                    encoded_title = re.sub(r'\s+', '+', rec['title'])
                    paper_url = f"https://scholar.google.com/scholar?q={encoded_title}"
                formatted_recommendations += f"[Access Paper]({paper_url})\n\n"
        except Exception as e:
            print(f"Error generating paper recommendations: {str(e)}")
            backup_prompt = ChatPromptTemplate.from_template(
                """
                Based on the following research paper content:
                
                {paper_content}
                
                Provide 5 relevant related research papers that might be of interest.
                For each paper, include:
                1. Title (a real paper title if possible)
                2. Authors
                3. Year
                4. Brief description of relevance
                5. MOST IMPORTANTLY: A direct URL to access the paper (use Google Scholar, arXiv, or ResearchGate)
                
                Format your response in markdown with clear headings and clickable links.
                Make sure every recommendation has a working URL.
                """
            )
            backup_chain = backup_prompt | gemini_model | StrOutputParser()
            formatted_recommendations = backup_chain.invoke({"paper_content": processing_text})
        return {
            "paper_id": paper_id,
            "filename": original_filename,
            "summary": summary,
            "recommendations": formatted_recommendations,
            "success": True
        }
    except Exception as e:
        print(f"Error processing research paper: {str(e)}")
        return {
            "paper_id": paper_id,
            "filename": original_filename,
            "error": str(e),
            "success": False
        }

# --------------------- Main Task Functions ---------------------
def run_research(topic: str, use_markdown_display: bool = True, db_path: str = "research_papers.db") -> Optional[Dict[str, Any]]:
    """Perform research on a specific topic"""
    if not topic.strip():
        print("Please enter a valid topic.")
        return
    print(f"\nResearching '{topic}'... This may take a moment.")
    try:
        initialize_database(db_path)
        print("- Generating detailed report...")
        report = generate_report(topic)
        print("- Finding related topics...")
        recommendations = generate_recommendations(topic)
        full_report = create_full_report(topic, report, recommendations)
        filename = save_markdown_file(topic, full_report)
        print("\n" + "="*50)
        print(f"RESEARCH REPORT: {topic.upper()}")
        print("="*50 + "\n")
        display_markdown(report, use_markdown_display)
        print("\n" + "="*50)
        print("RECOMMENDED RELATED TOPICS")
        print("="*50 + "\n")
        display_markdown(recommendations, use_markdown_display)
        print("\n" + "="*50)
        print(f"Full report saved to: {filename}")
        print("="*50)
        return {
            "report": report,
            "recommendations": recommendations,
            "full_report": full_report,
            "filename": filename
        }
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

def run_paper_analysis(file_path: str, original_filename: Optional[str] = None, use_markdown_display: bool = True, db_path: str = "research_papers.db") -> Optional[Dict[str, Any]]:
    """Analyze a research paper file"""
    try:
        initialize_database(db_path)
        result = process_research_paper(file_path, original_filename, db_path=db_path)
        if result["success"]:
            print("\n" + "="*50)
            print(f"PAPER ANALYSIS: {result['filename']}")
            print("="*50 + "\n")
            display_markdown(result["summary"], use_markdown_display)
            print("\n" + "="*50)
            print("RECOMMENDED RELATED PAPERS")
            print("="*50 + "\n")
            display_markdown(result["recommendations"], use_markdown_display)
            full_analysis = create_full_paper_analysis(
                result["filename"],
                result["summary"],
                result["recommendations"]
            )
            analysis_filename = save_markdown_file(f"paper_analysis_{sanitize_filename(result['filename'])}", full_analysis)
            print("\n" + "="*50)
            print(f"Full analysis saved to: {analysis_filename}")
            print("="*50)
            return {
                "summary": result["summary"],
                "recommendations": result["recommendations"],
                "full_analysis": full_analysis,
                "filename": analysis_filename
            }
        else:
            print(f"Failed to process paper: {result.get('error', 'Unknown error')}")
            return None
    except Exception as e:
        print(f"An error occurred while analyzing the paper: {str(e)}")
        return None

def run_web_interface(use_markdown_display: bool = True, db_path: str = "research_papers.db"):
    """Run a web interface using IPython widgets"""
    try:
        from ipywidgets import widgets
        from IPython.display import display, clear_output
        output = widgets.Output()
        topic_input = widgets.Text(description='Topic:', placeholder='Enter research topic')
        search_button = widgets.Button(description='Research Topic')
        file_upload = widgets.FileUpload(
            accept='.pdf,.docx,.doc',
            multiple=False,
            description='Upload Paper'
        )
        analyze_button = widgets.Button(description='Analyze Paper')
        def on_search_click(b):
            with output:
                clear_output()
                run_research(topic_input.value, use_markdown_display, db_path)
        def on_analyze_click(b):
            with output:
                clear_output()
                if not file_upload.value:
                    print("Please upload a research paper file (PDF or DOCX).")
                    return
                file_data = next(iter(file_upload.value.values()))
                file_name = next(iter(file_upload.value.keys()))
                with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(file_name)[1]) as temp_file:
                    temp_file.write(file_data['content'])
                    temp_path = temp_file.name
                try:
                    run_paper_analysis(temp_path, file_name, use_markdown_display, db_path)
                finally:
                    os.unlink(temp_path)
        search_button.on_click(on_search_click)
        analyze_button.on_click(on_analyze_click)
        tab1 = widgets.VBox([topic_input, search_button])
        tab2 = widgets.VBox([file_upload, analyze_button])
        tabs = widgets.Tab(children=[tab1, tab2])
        tabs.set_title(0, 'Topic Research')
        tabs.set_title(1, 'Paper Analysis')
        display(tabs)
        display(output)
    except ImportError:
        print("This function requires ipywidgets. Please install with: pip install ipywidgets")
        print("Running in command line mode instead.")
        run()

# --------------------- Helper Functions for Jupyter ---------------------
def research_topic(topic: str):
    """Helper function to research a topic directly from a Jupyter notebook"""
    return run_research(topic, use_markdown_display=True)

def analyze_paper(file_path: str):
    """Helper function to analyze a paper directly from a Jupyter notebook"""
    return run_paper_analysis(file_path, use_markdown_display=True)

# --------------------- Main Loop ---------------------
def run():
    """Main agent loop with support for both topic research and paper analysis"""
    initialize_database()
    print("🔍 AI Research Assistant Agent 🔍")
    print("--------------------------------")
    print("I can help you research topics and analyze research papers.")
    while True:
        print("\nWhat would you like to do?")
        print("1. Research a topic")
        print("2. Analyze a research paper")
        print("3. Exit")
        choice = input("Enter your choice (1-3): ")
        if choice == '1':
            topic = input("\nWhat topic would you like to research? ")
            if topic.strip():
                run_research(topic)
            else:
                print("Please enter a valid topic.")
        elif choice == '2':
            file_path = input("\nEnter the path to the research paper file (PDF or DOCX): ")
            if os.path.exists(file_path):
                run_paper_analysis(file_path)
            else:
                print(f"File not found: {file_path}")
        elif choice == '3':
            print("Thank you for using the AI Research Assistant. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

# --------------------- Execution ---------------------
if __name__ == "__main__":
    run()
